In [1]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import (
    FunctionTransformer,
    PowerTransformer,
    MinMaxScaler,
    OneHotEncoder,
    Binarizer,
    StandardScaler
)


from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error
from lightgbm import LGBMRegressor
from sklearn.decomposition import PCA


import mlflow
import mlflow.sklearn

from matplotlib import pyplot as plt
import seaborn as sns
import optuna

In [2]:
dataset_train = pd.read_csv('../data/processed/dataset_train.csv', index_col=0)
dataset_test = pd.read_csv('../data/processed/dataset_test.csv', index_col=0)

In [3]:
# column names
cols_event_embeddings = [f'event_feat_{i}' for i in range(768)]
cols_event_dummies = ['BACK STRAIN', 'CLEANING INSTRUMENTS', 'CLEANING LEFT SHOULDER SPLINTER', 'CUT WITH KNIFE', 'DEALING CARDS RIGHT TENDON SYNOVITIS', 'FELL', 'FELL FROM LADDER', 'FELL OFF LADDER', 'FRACTURED LEFT WRIST FRACTURE', 'GRINDING FOREIGN BODY', 'GRINDING STEEL BEAM INFECTION', 'HIT AIR HOSE', 'HIT ELBOW', 'HIT FLOOR SHEET', 'INJURY', 'INJURY LIFTING', 'INJURY LIFTING STRAIN', 'JAMMED RIGHT HAND PUNCTURE', 'KNIFE LACERATION', 'KNIFE SLIPPED', 'LACERATION', 'LEFT LEG SNAKE BITE', 'LIFTING', 'LIFTING BOXES', 'LIFTING PATIENT', 'LIFTING TYRES', 'LOWER BACK STRAIN', 'MOTOR VEHICLE COLLISION', 'PREPARING PIPES FOR PAINTING PUNCTURE', 'PULLING CABLES', 'REDBACK SPIDER BITE', 'SLIPPED AND FELL', 'SLIPPED ON FLOOR', 'SLIPPED ON ROLLER', 'SLIPPED ON STAIRS', 'SLIPPED ON WET FLOOR', 'SLIPPED USING LATHE', 'SORTING ALUMINIUM BARS STRAIN', 'SPRAINED RIGHT ANKLE FRACTURE', 'STRAIN', 'STRAIN LIFTING PARTS', 'STRAIN LIFTING STRAIN', 'STRUCK KNIFE LACERATED', 'STRUCK KNIFE LACERATION', 'STRUCK PALLET', 'STRUCK WITH KNIFE', 'USING AIR HOSE STRAIN', 'USING DRILL']
cols_body_parts_embeddings = [f'body_feat_{i}' for i in range(768)]

In [4]:
def train_cv(model, X, y, metric = 'rmse', kfold = 5):

    if metric != 'rmse':
        raise NotImplemented

    scores = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')

    mse_scores = -scores
    rmse_scores = np.sqrt(mse_scores)

    return rmse_scores.mean(), rmse_scores.std()

In [5]:
log_transformer = Pipeline([
    ('log', FunctionTransformer(np.log, validate=False)),
    ('scaler', StandardScaler()) 
])

# gender to bool 
def gender_to_bool(gender_column):
    """
    - Fill missing values with 'M'
    - Replace 'U' with 'M'
    - Return a boolean-ish column: 1 for 'M', 0 otherwise
    """
    g = pd.Series(gender_column.squeeze(), dtype=str).fillna('M').replace('U', 'M')
    is_male = (g == 'M').astype(int)
    return is_male.values.reshape(-1, 1)

gender_transformer = FunctionTransformer(gender_to_bool, validate=False)

# hours worked per week -> to buckets -> to one_hot
def bucket_hours_worked(dtt_array):
    return pd.cut(
        dtt_array.squeeze(), 
        bins=[-np.inf, 37, 41, np.inf],
        labels=["<=37", "37-41", ">41"]
    ).astype(str).values.reshape(-1, 1)

hours_worked_bucketer = FunctionTransformer(bucket_hours_worked, validate=False)
hours_worked_encoder = OneHotEncoder(drop='first')
hours_worked_pipeline = Pipeline([
    ('bucketizer', hours_worked_bucketer),
    ('encoder', hours_worked_encoder)
])

# DaysToReport (DateReported - DateTimeOfAccident) -> to buckets -> to one_hot
def bucket_days_to_report(dtt_array):
    return pd.cut(
        dtt_array.squeeze(), 
        bins=[-np.inf, 80, 300, 500, np.inf],
        labels=["<=80", "80-300", "300-500", ">500"]
    ).astype(str).values.reshape(-1, 1)

days_to_report_bucketer = FunctionTransformer(bucket_days_to_report, validate=False)
days_to_report_encoder = OneHotEncoder(drop='first')
days_to_report_pipeline = Pipeline([
    ('bucketizer', days_to_report_bucketer),
    ('encoder', days_to_report_encoder)
])

# DaysWorkedPerWeek -> 1 if equals 5, 0 in any other case 
def days_worked_binarize(days_array):
    # Ensure we handle arrays or DataFrames by squeezing to 1D
    days = days_array.squeeze()
    binarized = (days == 5).astype(int)
    # Return as 2D array: (n_samples x 1)
    return binarized.values.reshape(-1, 1) if isinstance(days, pd.Series) else binarized.reshape(-1, 1)

days_worked_transformer = FunctionTransformer(days_worked_binarize, validate=False)

In [6]:
preprocessor = ColumnTransformer(
    transformers=[
        ('log_inc', log_transformer, ['InitialIncurredCalimsCost', 'WeeklyWages']),
        ('minmax_scaler', MinMaxScaler(), ['Age', 'YearAccident']),
        ('gender_bool', gender_transformer, ['Gender']),
        ('hww_bool_onehot', hours_worked_pipeline, ['HoursWorkedPerWeek']),
        ('dtt_bool_onehot', days_to_report_pipeline, ['DaysToReport']),
        ('has_dependent_bool', Binarizer(threshold=0), ['DependentChildren']),
        ('worked_five_days_bool', days_worked_transformer, ['DaysWorkedPerWeek']),
        ('onehot', OneHotEncoder(drop='first'), ['MaritalStatus', 'PartTimeFullTime']),
        ('pca_event_embeddings', PCA(n_components=250), cols_event_embeddings),
        ('passthrough_event_dummies', 'passthrough', cols_event_dummies),
        ('pca_body_parts_embeddings', PCA(n_components=80), cols_body_parts_embeddings),
        ('std_scaler', StandardScaler(), ['event_median_cost'])
    ],
    remainder='drop'
)

In [ ]:
regressor_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('linear_model', LGBMRegressor(force_col_wise=True))
])

model = TransformedTargetRegressor(
    regressor=regressor_pipeline,
    transformer=PowerTransformer(method='box-cox', standardize=False)
)

In [8]:
dataset_train

,DateTimeOfAccident,DateReported,Age,Gender,MaritalStatus,DependentChildren,DependentsOther,WeeklyWages,PartTimeFullTime,HoursWorkedPerWeek,...,body_feat_758,body_feat_759,body_feat_760,body_feat_761,body_feat_762,body_feat_763,body_feat_764,body_feat_765,body_feat_766,body_feat_767
WC8285054,2002-04-09 07:00:00+00:00,2002-07-05 00:00:00+00:00,48.0,M,M,0.0,0.0,500.00,F,38.0,...,0.068680,0.025073,0.005782,-0.038884,0.058474,-0.020736,0.029686,-0.014542,-0.063414,0.004848
WC6982224,1999-01-07 11:00:00+00:00,1999-01-20 00:00:00+00:00,43.0,F,M,0.0,0.0,509.34,F,37.5,...,0.088765,0.020062,0.012920,-0.024838,0.031098,-0.021392,-0.002665,-0.011949,-0.061116,-0.003289
WC5481426,1996-03-25 00:00:00+00:00,1996-04-14 00:00:00+00:00,30.0,M,U,0.0,0.0,709.10,F,38.0,...,0.064991,0.032580,0.008295,-0.054072,0.052691,-0.002578,0.047455,-0.003457,-0.033384,-0.000423
WC9775968,2005-06-22 13:00:00+00:00,2005-07-22 00:00:00+00:00,41.0,M,S,0.0,0.0,555.46,F,38.0,...,0.070748,0.021465,0.012034,-0.043719,0.070600,0.006588,-0.030409,-0.009374,-0.076178,0.045278
WC2634037,1990-08-29 08:00:00+00:00,1990-09-27 00:00:00+00:00,36.0,M,M,0.0,0.0,377.10,F,38.0,...,0.079619,0.021140,0.043047,-0.006216,0.047252,-0.019845,0.000466,-0.033875,-0.028523,-0.011660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WC9370727,2004-08-21 18:00:00+00:00,2004-09-08 00:00:00+00:00,32.0,F,S,0.0,0.0,500.00,F,38.0,...,0.062220,0.018751,0.004911,-0.018006,0.007490,-0.031072,0.042807,-0.027719,-0.058011,-0.024113
WC8396269,2002-04-28 09:00:00+00:00,2002-09-03 00:00:00+00:00,20.0,F,S,0.0,0.0,500.00,F,40.0,...,0.077031,0.006508,0.023904,-0.006732,0.057064,-0.015958,0.007836,-0.026323,-0.068814,0.009221
WC3609528,1992-02-28 09:00:00+00:00,1992-03-18 00:00:00+00:00,19.0,M,S,0.0,0.0,283.00,F,40.0,...,0.049300,-0.024605,0.006685,-0.008855,0.019896,-0.023859,0.005520,-0.028199,-0.028116,-0.006528
WC5038565,1995-01-10 07:00:00+00:00,1995-01-31 00:00:00+00:00,24.0,M,S,0.0,0.0,200.00,F,38.0,...,0.051226,0.010259,-0.005054,-0.001850,-0.006115,-0.033084,0.029406,-0.032876,-0.057922,-0.025492


In [9]:

scores = cross_val_score(
    model, 
    dataset_train, 
    dataset_train['UltimateIncurredClaimCost'], 
    cv=5, 
    scoring='neg_mean_squared_error'
)
mse_scores = -scores
rmse_scores = np.sqrt(mse_scores)

print(f"CV MSE:  {mse_scores.mean():.3f}  (+/- {mse_scores.std():.3f})")
print(f"CV RMSE: {rmse_scores.mean():.3f}  (+/- {rmse_scores.std():.3f})")

[LightGBM] [Info] Total Bins 85015
[LightGBM] [Info] Number of data points in the train set: 43199, number of used features: 395
[LightGBM] [Info] Start training from score 5.909642


/Users/alexis/Projects/202502 Actuarial Loss Prediction/.env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Total Bins 85024
[LightGBM] [Info] Number of data points in the train set: 43199, number of used features: 395
[LightGBM] [Info] Start training from score 5.914034


/Users/alexis/Projects/202502 Actuarial Loss Prediction/.env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Total Bins 85026
[LightGBM] [Info] Number of data points in the train set: 43199, number of used features: 395
[LightGBM] [Info] Start training from score 5.945550


/Users/alexis/Projects/202502 Actuarial Loss Prediction/.env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Total Bins 85029
[LightGBM] [Info] Number of data points in the train set: 43199, number of used features: 395
[LightGBM] [Info] Start training from score 5.959808


/Users/alexis/Projects/202502 Actuarial Loss Prediction/.env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Total Bins 85028
[LightGBM] [Info] Number of data points in the train set: 43200, number of used features: 395
[LightGBM] [Info] Start training from score 5.940370
CV MSE:  571107743.684  (+/- 52125504.997)
CV RMSE: 23871.174  (+/- 1129.060)


/Users/alexis/Projects/202502 Actuarial Loss Prediction/.env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [10]:
X_train = dataset_train.drop(columns=['UltimateIncurredClaimCost'])
y_train = dataset_train['UltimateIncurredClaimCost']

def run_experiment(
        experiment_name, 
        run_name, 
        regressor_object, 
        kfold=5, 
        save_model=True
    ):
    """
    Runs an ML experiment with cross-validation and logs metrics to MLflow,
    tracking overall and segmented performance for targets above and below 100,000.

    Parameters:
    - experiment_name (str): Name of the MLflow experiment.
    - run_name (str): Name of the MLflow run.
    - regressor_object: The regression model.
    - kfold (int): Number of folds for cross-validation.
    - save_model (bool): Whether to save the trained model in MLflow.

    Logs:
    - Overall CV MSE, RMSE, and MAE
    - MSE, RMSE, and MAE for y_true > 100,000
    - MSE, RMSE, and MAE for y_true ≤ 100,000
    """

    # Define pipeline with preprocessing and model
    regressor_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('linear_model', regressor_object)
    ])

    model = TransformedTargetRegressor(
        regressor=regressor_pipeline,
        transformer=PowerTransformer(method='box-cox', standardize=False)
    )

    # Set MLflow experiment
    mlflow.set_experiment(experiment_name)

    with mlflow.start_run(run_name=run_name):
        # Log hyperparameters
        params = model.get_params(deep=True)
        for param_name, param_value in params.items():
            if isinstance(param_value, (str, int, float, bool, type(None))):
                mlflow.log_param(param_name, param_value)
            else:
                mlflow.log_param(param_name, str(param_value)[:5000])
        
        mlflow.log_param("n_features_in", X_train.shape[1])

        # Perform cross-validation with predictions
        y_pred = cross_val_predict(model, X_train, y_train, cv=kfold)

        # Compute overall errors
        overall_mse = mean_squared_error(y_train, y_pred)
        overall_rmse = np.sqrt(overall_mse)
        overall_mae = mean_absolute_error(y_train, y_pred)

        # Compute errors for high and low y_true
        high_mask = y_train > 100000

        high_mse = mean_squared_error(y_train[high_mask], y_pred[high_mask])
        high_rmse = np.sqrt(high_mse)
        high_mae = mean_absolute_error(y_train[high_mask], y_pred[high_mask])
        
        low_mse = mean_squared_error(y_train[~high_mask], y_pred[~high_mask])
        low_rmse = np.sqrt(low_mse)
        low_mae = mean_absolute_error(y_train[~high_mask], y_pred[~high_mask])
    
        print(f"RMSE Overall:  {overall_rmse:.3f} | MAE: {overall_mae:.3f}")
        print(f"RMSE for low y_true (<= 100k): {low_rmse:.3f} | MAE: {low_mae:.3f}")
        print(f"RMSE for high y_true (> 100k): {high_rmse:.3f} | MAE: {high_mae:.3f}")
        

        # Log overall metrics
        mlflow.log_metric("cv_mse", overall_mse)
        mlflow.log_metric("cv_rmse", overall_rmse)
        mlflow.log_metric("cv_mae", overall_mae)

        # Log segmented metrics
        mlflow.log_metric("cv_mse_high", high_mse)
        mlflow.log_metric("cv_rmse_high", high_rmse)
        mlflow.log_metric("cv_mae_high", high_mae)
        mlflow.log_metric("cv_mse_low", low_mse)
        mlflow.log_metric("cv_rmse_low", low_rmse)
        mlflow.log_metric("cv_mae_low", low_mae)

        # Fit final model on full dataset
        if save_model:
            model.fit(X_train, y_train)
            mlflow.sklearn.log_model(model, artifact_path="models")
    
    return overall_rmse


In [11]:

def objective(trial):
    """Objective function for Optuna to optimize XGBoost hyperparameters."""
    
    # Suggest hyperparameters to optimize
    params = {
        #"num_leaves": trial.suggest_int("num_leaves", 20, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.09, 0.4, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 10.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 10.0),
        "boosting_type": "gbdt",
        "objective": "regression",
        "force_col_wise": True,
        "metric": "rmse",
    }

    rmse = run_experiment(
        'Optuna LightGBM',
        'LightGBM optimization',
        LGBMRegressor(**params),
        kfold=5,
        save_model=False
    )

    return rmse


In [12]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=35)

# Print the best result
print(f"Best trial: {study.best_trial.params}")


[I 2025-03-02 18:43:25,613] A new study created in memory with name: no-name-81154639-d46b-4444-89d0-2c17f57139ad


[LightGBM] [Warning] lambda_l1 is set=7.942096909928619, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.942096909928619
[LightGBM] [Warning] lambda_l2 is set=2.977233533779348, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.977233533779348
[LightGBM] [Warning] lambda_l1 is set=7.942096909928619, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.942096909928619
[LightGBM] [Warning] lambda_l2 is set=2.977233533779348, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.977233533779348
[LightGBM] [Info] Total Bins 85015
[LightGBM] [Info] Number of data points in the train set: 43199, number of used features: 395
[LightGBM] [Info] Start training from score 5.909642


/Users/alexis/Projects/202502 Actuarial Loss Prediction/.env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=7.942096909928619, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.942096909928619
[LightGBM] [Warning] lambda_l2 is set=2.977233533779348, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.977233533779348
[LightGBM] [Warning] lambda_l1 is set=7.942096909928619, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.942096909928619
[LightGBM] [Warning] lambda_l2 is set=2.977233533779348, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.977233533779348
[LightGBM] [Warning] lambda_l1 is set=7.942096909928619, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.942096909928619
[LightGBM] [Warning] lambda_l2 is set=2.977233533779348, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.977233533779348
[LightGBM] [Info] Total Bins 85024
[LightGBM] [Info] Number of data points in the train set: 43199, number of used features: 395
[LightGBM] [Info] Start training from score 5.914034


/Users/alexis/Projects/202502 Actuarial Loss Prediction/.env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


...

[LightGBM] [Warning] lambda_l1 is set=1.220999422387717, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.220999422387717
[LightGBM] [Warning] lambda_l2 is set=7.232289905627115, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.232289905627115
[LightGBM] [Warning] lambda_l1 is set=1.220999422387717, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.220999422387717
[LightGBM] [Warning] lambda_l2 is set=7.232289905627115, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.232289905627115
[LightGBM] [Warning] lambda_l1 is set=1.220999422387717, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.220999422387717
[LightGBM] [Warning] lambda_l2 is set=7.232289905627115, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.232289905627115
[LightGBM] [Info] Total Bins 85029
[LightGBM] [Info] Number of data points in the train set: 43199, number of used features: 395
[LightGBM] [Info] Start training from score 5.959808


/Users/alexis/Projects/202502 Actuarial Loss Prediction/.env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=1.220999422387717, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.220999422387717
[LightGBM] [Warning] lambda_l2 is set=7.232289905627115, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.232289905627115
[LightGBM] [Warning] lambda_l1 is set=1.220999422387717, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.220999422387717
[LightGBM] [Warning] lambda_l2 is set=7.232289905627115, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.232289905627115
[LightGBM] [Warning] lambda_l1 is set=1.220999422387717, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.220999422387717
[LightGBM] [Warning] lambda_l2 is set=7.232289905627115, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.232289905627115
[LightGBM] [Info] Total Bins 85028
[LightGBM] [Info] Number of data points in the train set: 43200, number of used features: 395
[LightGBM] [Info] Start training from score 5.940370


/Users/alexis/Projects/202502 Actuarial Loss Prediction/.env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-03-02 18:58:28,857] Trial 34 finished with value: 23903.81890669048 and parameters: {'max_depth': 9, 'learning_rate': 0.09993424828472007, 'n_estimators': 100, 'min_child_weight': 6, 'subsample': 0.6967379799988431, 'colsample_bytree': 0.6479796728670014, 'lambda_l1': 1.220999422387717, 'lambda_l2': 7.232289905627115}. Best is trial 21 with value: 23853.022562628455.


[LightGBM] [Warning] lambda_l1 is set=1.220999422387717, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.220999422387717
[LightGBM] [Warning] lambda_l2 is set=7.232289905627115, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.232289905627115
RMSE Overall:  23903.819 | MAE: 5593.308
RMSE for low y_true (<= 100k): 9273.440 | MAE: 3219.726
RMSE for high y_true (> 100k): 164222.296 | MAE: 134677.207
Best trial: {'max_depth': 10, 'learning_rate': 0.09134879752884997, 'n_estimators': 200, 'min_child_weight': 4, 'subsample': 0.7406110320807305, 'colsample_bytree': 0.6527923965773857, 'lambda_l1': 1.5854842816691632, 'lambda_l2': 8.833659109656566}


In [13]:
import optuna.visualization as vis

fig = vis.plot_optimization_history(study)
fig.show()


In [15]:
fig = vis.plot_param_importances(study)
fig.show()


In [16]:
best_params = study.best_trial.params


regressor_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('linear_model', LGBMRegressor(**best_params))
])

model = TransformedTargetRegressor(
    regressor=regressor_pipeline,
    transformer=PowerTransformer(method='box-cox', standardize=False)
)

In [17]:
model.fit(X_train, y_train)

[LightGBM] [Warning] lambda_l1 is set=1.5854842816691632, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5854842816691632
[LightGBM] [Warning] lambda_l2 is set=8.833659109656566, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.833659109656566
[LightGBM] [Warning] lambda_l1 is set=1.5854842816691632, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5854842816691632
[LightGBM] [Warning] lambda_l2 is set=8.833659109656566, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.833659109656566
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018197 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 85037
[LightGBM] [Info] Number of data points in the train set: 53999, number of used features: 395
[LightGBM] [Info] Start training from score 5.933758


TransformedTargetRegressor(regressor=Pipeline(steps=[('preprocessor',
                                                      ColumnTransformer(transformers=[('log_inc',
                                                                                       Pipeline(steps=[('log',
                                                                                                        FunctionTransformer(func=<ufunc 'log'>)),
                                                                                                       ('scaler',
                                                                                                        StandardScaler())]),
                                                                                       ['InitialIncurredCalimsCost',
                                                                                        'WeeklyWages']),
                                                                                      ('minmax_scaler',
                                                                                       MinMaxScaler(),
                                                                                       ['Age',
                                                                                        'YearAccident']),
                                                                                      ('gender_bool',
                                                                                       FunctionTransformer(func=<fun...
                                                                                      ('std_scaler',
                                                                                       StandardScaler(),
                                                                                       ['event_median_cost'])])),
                                                     ('linear_model',
                                                      LGBMRegressor(colsample_bytree=0.6527923965773857,
                                                                    lambda_l1=1.5854842816691632,
                                                                    lambda_l2=8.833659109656566,
                                                                    learning_rate=0.09134879752884997,
                                                                    max_depth=10,
                                                                    min_child_weight=4,
                                                                    n_estimators=200,
                                                                    subsample=0.7406110320807305))]),
                           transformer=PowerTransformer(method='box-cox',
                                                        standardize=False))

In [18]:
y_test_pred = model.predict(dataset_test)
y_test_pred

[LightGBM] [Warning] lambda_l1 is set=1.5854842816691632, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5854842816691632
[LightGBM] [Warning] lambda_l2 is set=8.833659109656566, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.833659109656566


/Users/alexis/Projects/202502 Actuarial Loss Prediction/.env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but LGBMRegressor was fitted with feature names



array([ 6976.63844136,  2392.09955833, 20966.5537356 , ...,
        7474.87164317,  7381.86565926,   380.62672658], shape=(36000,))

In [19]:
submission_df = pd.DataFrame(zip(dataset_test.index, y_test_pred), columns=['ClaimNumber', 'UltimateIncurredClaimCost'])

print(submission_df.shape)
submission_df.head()

(36000, 2)


,ClaimNumber,UltimateIncurredClaimCost
0,WC8145235,6976.638441
1,WC2005111,2392.099558
2,WC6899143,20966.553736
3,WC5502023,308.760132
4,WC4785156,2855.345134


In [20]:
submission_df.to_csv('../data/output/submission_lgbm.csv', index=False)